### Step 1: Install necesscary packages

In [1]:
!pip install matplotlib
# !pip install torch numpy transformers datasets tiktoken wandb tqdm
!pip install numpy transformers datasets tiktoken wandb tqdm


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.1 MB 13.4 MB/s eta 0:00:01
   ------------------------- -------------- 5.2/8.1 MB 13.9 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 13.2 MB/s  0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 14.3 MB/s  0:00:00

   ------------- -------------------------- 2/6 [fonttools]
   ------------- -------------------------- 2/6 [fonttools]
   ------------- -------------------------- 2/6 [fonttools]
   ------------- -------------------------- 2/6 [fonttools]
   ------------- -------------------------- 2/6 [fonttools]
   ------------- -------------------------- 2/6 [fonttools]
   --------------------------------- ------ 5/6 [matplotlib]
   --------------------------------- ------ 5/6 [matplotlib]
   --------------------------------- ------ 5/6 [matplotlib]
   -------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires sympy==1.13.1, but you have sympy 1.14.0 which is incompatible.


### Step 2: Package imports and configuration

In [109]:
import sys
import os
sys.path.append(os.path.abspath("..")) 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
from model import GPT, GPTConfig
import random
from tqdm import tqdm
import time
import json
import matplotlib.pyplot as plt
# Configuration
beta = 0.5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
base_lr = 1e-4
epochs = 5
batch_size = 64
max_length =64
num_samples = 1
max_new_tokens = 200
temperature = 0.8
top_k = 200
# tokenizer
# # tokenizer
# with open("../sft/meta.pkl", "rb") as f:
#     meta = pickle.load(f)
# stoi, itos = meta["stoi"], meta["itos"]
# #def encode(s): return [stoi[c] for c in s]
# #def decode(l): return ''.join([itos[i] for i in l])
# PAD_IDX = 0
# UNK_IDX = stoi.get("<unk>", stoi.get(" ", PAD_IDX))  # prefer <unk>, then space, else pad(0)

# def encode(s: str):
#     # map unseen characters to UNK instead of raising KeyError
#     return [stoi.get(c, UNK_IDX) for c in s]

# def decode(ids):
#     return ''.join(itos[i] for i in ids if 0 <= i < len(itos))
# with open("../sft/meta.pkl", "rb") as f:
#     meta = pickle.load(f)
# stoi, itos = meta["stoi"], meta["itos"]
# def encode(s): return [stoi[c] for c in s if c in stoi] # I EDITED THIS LINE
# def decode(l): return ''.join([itos[i] for i in l])
# tokenizer
with open("../sft/meta.pkl", "rb") as f:
    meta = pickle.load(f)
stoi, itos = meta["stoi"], meta["itos"]
#def encode(s): return [stoi[c] for c in s]
#def decode(l): return ''.join([itos[i] for i in l])
PAD_IDX = 0
UNK_IDX = stoi.get("<unk>", stoi.get(" ", PAD_IDX))  # prefer <unk>, then space, else pad(0)

def encode(s: str):
    # map unseen characters to UNK instead of raising KeyError
    return [stoi.get(c, UNK_IDX) for c in s]

def decode(ids):
    return ''.join(itos[i] for i in ids if 0 <= i < len(itos))


### Step 3: Define helper functions

In [110]:
def compute_logprob(input_ids):
    inputs = input_ids[:, :-1]
    targets = input_ids[:, 1:]
    logits, _ = gpt(inputs, full_seq=True)
    B, T, V = logits.size()
    logits_flat = logits.reshape(-1, V)
    targets_flat = targets.reshape(-1)
    loss = F.cross_entropy(logits_flat, targets_flat, ignore_index=0, reduction='none')
    loss = loss.reshape(B, T)
    attention_mask = (targets != 0).float()
    loss = (loss * attention_mask).sum(dim=1) / attention_mask.sum(dim=1)
    return -loss 

def pad_or_truncate(seq, max_length):
    return seq[-max_length:] if len(seq) > max_length else seq + [0] * (max_length - len(seq))

def get_batches(lines, batch_size):
    random.shuffle(lines)   
    #for l in lines:
    #    print(l[1])
    for i in range(0, len(lines), batch_size):
        batch = lines[i:i+batch_size]
        if len(batch) < batch_size:
            continue
        neg_inputs = [pad_or_truncate(encode(p['negative'] + '\n\n\n\n'), max_length) for p in batch]
        pos_inputs = [pad_or_truncate(encode(p['positive'] + '\n\n\n\n'), max_length) for p in batch]
        neg_tensor = torch.tensor(neg_inputs, dtype=torch.long, device=device)
        pos_tensor = torch.tensor(pos_inputs, dtype=torch.long, device=device)
        yield neg_tensor, pos_tensor

### Step 4: Load the pretrained NanoGPT model

In [111]:
ckpt = torch.load("../sft/gpt.pt", map_location=device)
gptconf = GPTConfig(**ckpt['model_args'])
gpt = GPT(gptconf)
state_dict = ckpt['model']
unwanted_prefix = '_orig_mod.'
for k in list(state_dict.keys()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
gpt.load_state_dict(state_dict)
gpt.to(device).train()


# ### KIERAN ADDED THIS, REMEBER TO REMOVE BEFORE SUBMITTING
print(torch.cuda.is_available())
import torch
print(1212,torch.version.cuda)
print(device)
print(torch.cuda.is_available())
print("Model first parameter device:", next(gpt.parameters()).device)


C:\Users\rkuru\AppData\Local\Temp\ipykernel_34224\3578242387.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("../sft/gpt.pt", map_location=device)


True
1212 12.1
cuda
True
Model first parameter device: cuda:0


### Step 5: Load Data (**students are required to complete this part!**)

In [112]:
# Load data from ./data/pos_neg_pairs.json
import json
import tiktoken
# Loading the json file, CHANGE ADDRESS IF NEEDED
with open("../dpo/pos_neg_pairs.json", "r", encoding = "utf-8") as f:
    lines = json.load(f)

print(f"Loaded {len(lines)} pairs.")



Loaded 100000 pairs.


### Step 6: Build the optimizer and scheduler (**students are required to complete this part!**)

In [ ]:
# import math
# import torch
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import LambdaLR

# # ---------- AdamW with proper weight decay ----------
# wd = 0.1            # weight decay for weights (not biases/norm/embeddings)
# base_lr = 1e-4      # your chosen base LR
# betas = (0.9, 0.95)
# eps = 1e-8
# max_grad_norm = 1.0

# decay, no_decay = [], []
# for n, p in gpt.named_parameters():
#     if not p.requires_grad:
#         continue
#     is_bias = n.endswith(".bias")
#     is_norm = ("norm" in n.lower()) or ("ln" in n.lower())
#     is_embed = "embed" in n.lower()
#     if is_bias or is_norm or is_embed:
#         no_decay.append(p)
#     else:
#         decay.append(p)

# optimizer = AdamW(
#     [
#         {"params": decay, "weight_decay": wd},
#         {"params": no_decay, "weight_decay": 0.0},
#     ],
#     lr=base_lr,
#     betas=betas,
#     eps=eps,
# )

# # ---------- Warmup + Cosine scheduler ----------
# steps_per_epoch = len([1 for _ in range(0, len(lines), batch_size)])  # or len(lines)//batch_size
# total_steps = epochs * steps_per_epoch
# warmup_steps = max(1, int(0.1 * total_steps))  # 10% warmup (tweak as you like)

# def lr_lambda(step):
#     if step < warmup_steps:
#         return float(step) / float(max(1, warmup_steps))
#     # cosine decay from 1.0 -> 0.0 over the remaining steps
#     progress = (step - warmup_steps) / float(max(1, total_steps - warmup_steps))
#     return 0.5 * (1.0 + math.cos(math.pi * progress))

# scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)
# print(f"Total steps: {total_steps}, Warmup steps: {warmup_steps}")


# THIS IS THEIRS 

In [ ]:
# Configuration 
import math 
weight_decay = 1e-3
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0
decay_lr = True
max_iters = (len(lines) // batch_size) * epochs
warmup_iters =  int(0.1 * max_iters)

lr_decay_iters = max_iters
base_lr =  6e-4
min_lr = base_lr / 10


# optimizer = torch.optim.AdamW(gpt.parameters(), lr=base_lr, weight_decay=weight_decay, betas=(beta1, beta2))
decay_params = []
no_decay_params = []

for name, param in gpt.named_parameters():
    if param.requires_grad:
        # Don't apply weight decay to biases and layer norms
        if 'bias' in name or 'ln' in name or 'layernorm' in name:
            no_decay_params.append(param)
        else:
            decay_params.append(param)

optimizer = torch.optim.AdamW([
    {'params': decay_params, 'weight_decay': 1e-2},
    {'params': no_decay_params, 'weight_decay': 0.0}
], lr=base_lr, betas=(beta1, beta2))

# Learning rate decay scheduler (cosine with warmup) 
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return base_lr * (it + 1) / (warmup_iters + 1)
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff ranges 0..1
    return min_lr + coeff * (base_lr - min_lr)

### Step 7: Begin training (**students are required to complete this part!**)

In [136]:
# # can implement mixed precision training here (also implemented in the nanogpt train.py, i copied code here)

# """
# What is Mixed Precision Training? (sorry copied from chatgpt but i thinkk it makes sense?? it sped up the training time considerably)

# Normally, training uses 32‑bit floats (FP32) for everything.
# Mixed precision means:
# - Use 16‑bit floats (FP16 or BF16) for most forward & backward ops (faster + less memory).
# - Keep a few things in 32‑bit to avoid numerical instability (like the master weights in optimizer).
# - This gives ~2–3× training speedups on GPUs and lets you fit bigger batches.
# """
# from contextlib import nullcontext

# device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast

# # use bf16 if the gpu supports it, else fall back to fp16
# dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
# ptDtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]

# # float32 (full precision, 32-bit): default in pytorch, stable, but memory-heavy and slower on gpus
# # float16 / bfloat16 (half precision, 16-bit): uses half the memory, runs way faster on gpus
# # - but downside: less numerical precision, risk of underflow / overflow

# # hence: amp (automatic mixed precision) : use half precision when possible + fallback to float32 when needed
# # - matrix multiplication -> good in float16 (fast on gpu)
# # - loss, reductions -> keep in float32 (stable)
# # (dynamically decided per operation)
# ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptDtype)
# # Initialize GradScaler
# scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# # special case: tf32 settings (not a precision type explicitly picked, more of a backend optimisation)
# # these flags affect matrix multiplication and convolutions
# torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
# torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
# # if running on fp32 outcast, python is allowed to use tf32 internally


# # fp16 leads to gradient underflow
# # fp16 has a much smaller range than fp32
# # - fp32 can represent numbers as small as ~10^-38
# # - fp16: can represent numbers as small as ~6x10^-8
# # during backpropogation, gradients get very small. in fp16, these tiny gradients underflow to 0 -> model stops learning

# # solution: dynamic loss scaling using gradscaler
# # gradscaler cycle:
# # - scaler.scale(loss) → multiply loss by a scale factor (e.g., 1024, 2048)
# # - .backward() → gradients are now scaled up (less likely to underflow)
# # - scaler.unscale_(optimizer) → divide gradients back down to their true values
# # - scaler.step(optimizer) → optimizer uses the unscaled gradients
# # - scaler.update() → adjust the scale factor for next iteration
# scaler = torch.cuda.amp.GradScaler('cuda',enabled=(dtype == 'float16'))

In [137]:
# total_steps = len(lines) // batch_size
# global_step = 0
# for epoch in range(epochs):
#     pbar = tqdm(get_batches(lines, batch_size))
#     for step, (neg_tensor,pos_tensor) in enumerate(pbar):
#         ###########################################################

#         # We first zero the gradients to avoid accumulation so that we can correctly compute the gradients for this step
#         optimizer.zero_grad()

#         # We calculate the log-probabilities
#         neg_logprob = compute_logprob(neg_tensor)
#         pos_logprob = compute_logprob(pos_tensor)

#         # We then calculate the loss of the DPO by the formula where we take the mean of the individual losses
#         loss = -F.logsigmoid((pos_logprob - neg_logprob) * beta).mean()
        
#         # We then backpropagate the loss
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(gpt.parameters(), grad_clip)

#         lr = get_lr(global_step)
#         for param_group in optimizer.param_groups:
#             param_group['lr'] = lr
#         optimizer.step()
#         global_step += 1
        

#         # We update the progress bar with the current epoch, step, and loss
#         pbar.set_description(f"Epoch {epoch + 1} Step {step + 1} Loss {loss.item():.4f}")
#         ###########################################################
#     ckpt_path = f"./dpo.pt"
#     torch.save({
#         "model_state_dict": gpt.state_dict(),
#         "model_args": ckpt['model_args'],
#     }, ckpt_path)
#     print(f"Saved checkpoint to {ckpt_path}")
global_step = 0
for epoch in range(epochs):
    pbar = tqdm(get_batches(lines, batch_size))
    for step, (neg_tensor, pos_tensor) in enumerate(pbar):
        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        neg_logprob = compute_logprob(neg_tensor)
        pos_logprob = compute_logprob(pos_tensor)
        loss = -F.logsigmoid((pos_logprob - neg_logprob) * beta).mean()
        
        # Backward pass
        loss.backward()
        
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(gpt.parameters(), grad_clip)
        
        # Update learning rate BEFORE optimizer step
        lr = get_lr(global_step)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        
        # Optimizer step
        optimizer.step()
        
        # Update progress bar
        pbar.set_description(f"Epoch {epoch + 1}/{epochs} | Step {global_step} | Loss {loss.item():.4f} | LR {lr:.2e}")
        
        # Increment global step counter
        global_step += 1
    
    # Save checkpoint ONCE per epoch
    ckpt_path = f"./dpo_epoch_{epoch+1}.pt"
    torch.save({
        "model_state_dict": gpt.state_dict(),
        "model_args": ckpt['model_args'],
    }, ckpt_path)
    print(f"Saved checkpoint to {ckpt_path}")

Epoch 1/5 | Step 1561 | Loss 0.0000 | LR 9.31e-05: : 1562it [02:03, 12.68it/s]


Saved checkpoint to ./dpo_epoch_1.pt


Epoch 2/5 | Step 3123 | Loss 0.0000 | LR 7.10e-05: : 1562it [02:03, 12.67it/s]


Saved checkpoint to ./dpo_epoch_2.pt


Epoch 3/5 | Step 4685 | Loss 0.0000 | LR 4.25e-05: : 1562it [02:03, 12.63it/s]


Saved checkpoint to ./dpo_epoch_3.pt


Epoch 4/5 | Step 6247 | Loss 0.0000 | LR 1.90e-05: : 1562it [02:03, 12.68it/s]


Saved checkpoint to ./dpo_epoch_4.pt


Epoch 5/5 | Step 7809 | Loss 0.0000 | LR 1.00e-05: : 1562it [02:03, 12.66it/s]

Saved checkpoint to ./dpo_epoch_5.pt


In [138]:
# gpt.train()
# total_steps = len(lines) // batch_size
# for epoch in range(epochs):
#     pbar = tqdm(get_batches(lines, batch_size))
#     for step, (neg_tensor,pos_tensor) in enumerate(pbar):
#         ###########################################################
#         # ---- forward: log-probs for positive and negative samples ----
#         pos_logprob = compute_logprob(pos_tensor)   # [B]
#         neg_logprob = compute_logprob(neg_tensor)   # [B]

#         # ---- loss: preference-style (DPO-ish) + small anchor on positives ----
#         preference_term = -F.logsigmoid((pos_logprob - neg_logprob) / beta).mean()
#         anchor_term      = -0.10 * pos_logprob.mean()   # optional; set to 0.0 to disable
#         loss = preference_term + anchor_term

#         # ---- backward + step ----
#         optimizer.zero_grad(set_to_none=True)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(gpt.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()

#         # ---- progress bar ----
#         if (step + 1) % 50 == 0:
#             pbar.set_postfix({
#                 "loss": f"{loss.item():.4f}",
#                 "pref": f"{preference_term.item():.4f}",
#                 "lr": f"{scheduler.get_last_lr()[0]:.2e}",
#             })
#         ###########################################################
#     ckpt_path = f"./dpo.pt"
#     torch.save({
#         "model_state_dict": gpt.state_dict(),
#         "model_args": ckpt['model_args'],
#     }, ckpt_path)
#     print(f"Saved checkpoint to {ckpt_path}")

# THIS I STHIERS


### Step 8: Begin testing (**students are required to complete this part!**)

In [140]:
# Load the fine-tuned model
ckpt_path = "../dpo/dpo_epoch_5.pt"
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
gpt = GPT(gptconf).cuda()
try:
    state_dict = checkpoint['model']
except:
    state_dict = checkpoint['model_state_dict']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
gpt.load_state_dict(state_dict)
# Test
gpt.eval()
test_set = ["17+19=?", "3*17=?", "72/4=?", "72-x=34,x=?", "x*11=44,x=?", "3*17=?", "72/4=?", "72-x=34,x=?"]
with torch.no_grad():
    for prompt in test_set: 
        prompt_ids = encode(prompt)
        ###########################################################
        # Please complete the test code here!
        # This part i gpt generated could be wrong, couldnt find this in train.py lol 
        # Encode text → tensor
        prompt_ids = encode(prompt)
        x = torch.tensor(prompt_ids, dtype=torch.long, device=device).unsqueeze(0)  # [1, len(prompt)]

        # Generate continuation
        out = gpt.generate(
            x,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k
        )

        # Convert back to text
        generated_tokens = out[0][0].cpu().tolist()

        # Split into prompt + continuation
        prompt_len = len(prompt_ids)
        full_text = decode(generated_tokens)
        continuation = decode(generated_tokens[prompt_len:])

        print(f"Prompt: {prompt}")
        print(f"Answer: {continuation.strip()}\n")
        ###########################################################

C:\Users\rkuru\AppData\Local\Temp\ipykernel_34224\1280974629.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


Prompt: 17+19=?
Answer: eee e anZe The is ooe -8   tkequnZ oeZ oedZ 106 oedZ  oequrej oequ onZ onZ oedZ oe,edZ -Z oedZ 6Z oe,edZ +Z 9*-2+Zre,e?Z oe,eredZ oe,edZ oe,e,e,e onZ oedZ onZ oewedZ onZ onZ owe,e,e onZ owe180=Z oe,e,

Prompt: 3*17=?
Answer: eee e anZe is yea 9 uoe oedZ -97 e,e5 equredZ   equreqoonZ  onZre,edZ 90-6Z oe oedZ onZ oe,e,e.

Prompt: 72/4=?
Answer: eee e oectnZ seca se 14  oequaure  oequ/1160.

Prompt: 72-x=34,x=?
Answer: eeee anZe TIej  eq oe oequanZ  oeqonZ /+1*7.

Prompt: x*11=44,x=?
Answer: eeee anZe is 5 equoe te 80+9 equals 111  11.

Prompt: 3*17=?
Answer: eee e anZe keq ooe oe0Z oedZ re oequalnZ 1+Z oeredZ onZ oe,e onZ oe,e.

Prompt: 72/4=?
Answer: eee e T tnZ tec sequnZ  oedZ 1-1 oe,equredZ oeq0.

Prompt: 72-x=34,x=?
Answer: eeee anZe TIev oere is 6=Z equanZ 9.

